In [3]:
import time
import requests
import sqlite3
import pandas as pd
import re
import pytz
from datetime import datetime, timezone
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument('--headless=new')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True

# Import the data
df = pd.read_csv('Resources/google_cba.csv')

# from google.colab import drive
# drive.mount('/content/drive')

# Create a variable, `path`, containing the path to the `commonwealth_bank.db`
path = 'Resources/commonwealth_bank.db'


while True:
    # SQLite connection and cursor
    conn = sqlite3.connect(path)
    cursor = conn.cursor()

    # Create the table if it doesn't exist
    cursor.execute('''CREATE TABLE IF NOT EXISTS stock_data
                      (Stock_Symbol TEXT, Share_Price REAL, Price_Changing TEXT, Percent_Change_Value TEXT, Time DateTime)''')
    conn.commit()

    quote = 'CBA'

    baseurl = 'http://google.com/finance/quote/'
    URL = baseurl + quote + ":ASX?hl=en%27"
 
    page = requests.get(URL)
    
    soup = BeautifulSoup(page.text, 'html.parser')

    def price_changes(url):
        service = Service("/usr/bin/chromedriver")
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.get(url)
        time.sleep(5)
        html = driver.page_source
        driver.quit()

        soup = BeautifulSoup(html, "html.parser")

        element = soup.select_one('div.enJeMd')

        # page = requests.get(url)
        # soup = BeautifulSoup(html, 'html.parser')
        # element = soup.select_one('div.enJeMd')

        if element is not None:
            price_percent_changes = element.text.strip()
            price_n_percent_changes = re.findall(r'[-+]?\d*\.\d+%?', price_percent_changes)
            price_changing = price_n_percent_changes[1]
            percent_change_value = price_n_percent_changes[0]

            return price_changing, percent_change_value
        else:
            return None, None
     
    results = soup.find(class_="YMlKec fxKbKc")

    result = results.__str__()

    res = result.split("$")[1].split("<")[0]

    share_price = float(res.replace(",", ""))

    price_changing, percent_change_value = price_changes(URL)

    # Compare with existing data and update if new
    cursor.execute('''SELECT Share_Price FROM stock_data ORDER BY Time DESC LIMIT 1''')
    existing_share_price = cursor.fetchone()

    if existing_share_price is None or existing_share_price[0] != share_price:
        utc_dt = datetime.now(timezone.utc)
        au = pytz.timezone('Australia/Melbourne')
        current_time = utc_dt.astimezone(au).strftime('%d %b %y %H:%M:%S')

        cursor.execute('''INSERT INTO stock_data (Stock_Symbol, Share_Price, Price_Changing, Percent_Change_Value, Time)
                          VALUES (?, ?, ?, ?, ?)''', (quote, share_price, price_changing, percent_change_value, current_time))
        conn.commit()

        print("New data added to SQLite at timestamp:", current_time)
    
    else:
        print("Value does not match expected format.")
     
    cursor.execute('''SELECT * FROM stock_data''')
    df1 = pd.DataFrame(cursor.fetchall())
    
    path1 = 'Resources/google_cba.csv'
    with open(path1, 'a', encoding = 'utf-8-sig') as f:
      df1.to_csv(f)

    time.sleep(60)  # Wait for 1 minutes before checking again
    
  
# Close the SQLite connection
conn.close()


c:\Users\John Paul\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  


ValueError: could not convert string to float: 'None'